From the outputs of *"Execution Time for original data format.js"*, we see that the running time for the daily quests vary from 17 seconds to 24 seconds. We need another data representation to speed up the execution time.

After analyzing our query, we can see that our query can be divided into two parts: Finding "asin" for the required products in transaction_data, then look-up the product information in meta_data. Therefore, we are wondering if storing the information in transaction data would speed-up our query.

In this file, I will do the tasks mentioned above.

In [1]:
## Import required libraries:
import json
import pandas as pd
import random
import numpy as np
from faker import Faker
from datetime import timedelta, datetime
from bson import ObjectId

import os

In [2]:
file_paths = {
    "review": "generated data/review_data.json",
    "meta": "generated data/meta_data.json",
}

# Load each JSON file using json.load() and convert directly to pandas DataFrame
review_df = pd.read_json(file_paths["review"], lines=True, encoding='utf-8')
meta_df = pd.read_json(file_paths["meta"], lines=True, encoding='utf-8')

In [3]:
# We also read the sample data files:
file_paths_sample = {
    "review": "generated data/review_data_sample.json",
    "meta": "generated data/meta_data_sample.json",
    "user": "generated data/user_data_sample.json",
    "transaction": "generated data/transaction_data_sample.json"
}

# Load each JSON file using json.load() and convert directly to pandas DataFrame
review_sample_df = pd.read_json(file_paths_sample["review"], lines=True, encoding='utf-8')
meta_sample_df = pd.read_json(file_paths_sample["meta"], lines=True, encoding='utf-8')
user_sample_df = pd.read_json(file_paths_sample["user"], lines=True, encoding='utf-8')
transaction_sample_df = pd.read_json(file_paths_sample["transaction"], lines=True, encoding='utf-8')

In [3]:
# To check if the dataset is correctly loaded
print(review_df.shape)
print(meta_df.shape)
print(review_df.iloc[945])
print(meta_df.iloc[945])

(6460965, 11)
(1210967, 18)
overall                                                           4
vote                                                              1
verified                                                      False
reviewTime                                              08 14, 2000
unixReviewTime                                            966211200
reviewerID                                           A19646YDU8IH1I
reviewerName                                    Robert Ian Farquhar
asin                                                     B00000DMA8
style                                     {'Edition:': ' Standard'}
reviewText        Okay I admit it, the two main reasons I bought...
summary                                                   Good Fun!
Name: 945, dtype: object
asin                                                      B00001XDVT
title                                  Armorines: Project S.W.A.R.M.
feature            [Great Condition, cleaned and tested, *Car

In [5]:
# Merge review_df and meta_df on 'asin' to obtain price for each review
payment_methods = ['MasterCard', 'Visa', 'PayPal', 'Discover', 'Amex', 'Bitcoin']

merged_df = review_df.merge(meta_df[['asin', 'price']], on='asin', how='left')
valid_prices = merged_df['price'].str.startswith('$', na=False)
merged_df = merged_df[valid_prices]
merged_df['price'] = merged_df['price'].str[1:].astype(float)

merged_df['copy'] = [random.randint(1, 10) for _ in range(len(merged_df))]
merged_df['totalPrice'] = merged_df['price'] * merged_df['copy']

seconds_in_a_day = 86400
random_seconds = [random.randint(0, 7 * seconds_in_a_day) for _ in range(len(merged_df))]
review_times = pd.to_datetime(merged_df['unixReviewTime'], unit='s')
transaction_times = review_times - pd.to_timedelta(random_seconds, unit='s')
merged_df['transactionTime'] = transaction_times


merged_df['paymentMethod'] = [random.choice(payment_methods) for _ in range(len(merged_df))]
merged_df['transactionID'] = [str(ObjectId()) for _ in range(len(merged_df))]

# Adding "Overall" Column
transaction_df = merged_df[['transactionID', 'transactionTime', 'asin','reviewerID', 'copy', 'totalPrice', 'paymentMethod', 'overall']]

In [5]:
# Transform sample transaction data
merged_df = pd.merge(transaction_sample_df, meta_sample_df[['asin', 'title', 'price', 'also_buy', 'also_view']], on='asin', how='left')

# We'll create a new column 'product_info' which will be a dictionary of the required fields
merged_df['product_info'] = merged_df.apply(lambda row: {
    'asin': row['asin'],
    'title': row['title'],
    'price': row['price'],
    'also_buy': row['also_buy'],
    'also_view': row['also_view']
}, axis=1)

merged_df.drop(columns=['asin','title', 'description', 'price', 'also_buy', 'also_view'], inplace=True)

print(merged_df.head())  # Check the results

output_dir = "G:\\DSA5104 Project\\generated data"
merged_df.to_json(os.path.join(output_dir, 'transaction_data_sample_t.json'), orient='records', lines=True)

              transactionID  transactionTime      reviewerID  copy  \
0  65463da2e6e35c4ffca6d932    1098525317000  A36Y3ZMIWOV2DA     7   
1  65463da2e6e35c4ffca6d933    1098679454000  A35R5A5PN4SZUL     7   
2  65463da2e6e35c4ffca6d934    1098298562000  A2PXOP8AOGXQCO     7   
3  65463da2e6e35c4ffca6d935    1098414197000  A1WX4CIGZ6ZHNG     3   
4  65463da2e6e35c4ffca6d936    1519082412000  A3HWCOEZBA8Z23     9   

   totalPrice paymentMethod                                       product_info  
0      328.23        PayPal  {'asin': 'B00004SQPD', 'title': 'PlayStation 2...  
1      328.23      Discover  {'asin': 'B00004SQPD', 'title': 'PlayStation 2...  
2      328.23          Visa  {'asin': 'B00004SQPD', 'title': 'PlayStation 2...  
3      140.67    MasterCard  {'asin': 'B00004SQPD', 'title': 'PlayStation 2...  
4      422.01          Amex  {'asin': 'B00004SQPD', 'title': 'PlayStation 2...  


In MongoDB, we also received a warning: Some of the imported documents contained unbounded arrays that may degrade efficiency.

To deal with this problem, I use these two methods:
1. do not include descriptions in 'product_info' fields
2. limit the length of 'also_view' and 'also_buy' to 15 elements (elements exceed 15 will be ignored)

In [6]:
# Then, we apply the codes above to the whole dataset:
merged_df = pd.merge(transaction_df, meta_df[['asin', 'title', 'price', 'also_buy', 'also_view']], on='asin', how='left')

# Limit the 'also_buy' and 'also_view' arrays to the first 15 elements
def limit_array_length(row, field_name):
    return row[field_name][:15] if isinstance(row[field_name], list) else row[field_name]

merged_df['product_info'] = merged_df.apply(lambda row: {
    'asin': row['asin'],
    'title': row['title'],
    'price': row['price'],
    'also_buy': limit_array_length(row, 'also_buy'),
    'also_view': limit_array_length(row, 'also_view')
}, axis=1)

merged_df.drop(columns=['asin', 'title', 'price', 'also_buy', 'also_view'], inplace=True)
print(merged_df.head())  # Check the results


              transactionID     transactionTime      reviewerID  copy  \
0  654e19d0e6e35c3d6c3cb471 2015-10-13 05:24:01  A1HP7NVNPFMA4N     3   
1  654e19d0e6e35c3d6c3cb472 2015-07-26 07:21:03  A1JGAP0185YJI6     6   
2  654e19d0e6e35c3d6c3cb473 2015-02-19 16:02:56  A1YJWEXHQBWK2B     2   
3  654e19d0e6e35c3d6c3cb474 2015-02-14 21:16:41  A2204E1TH211HT     1   
4  654e19d0e6e35c3d6c3cb475 2014-12-23 23:10:52  A2RF5B5H74JLPE     1   

   totalPrice paymentMethod  overall  \
0       23.85          Visa        5   
1       47.70          Visa        4   
2       15.90          Visa        3   
3        7.95       Bitcoin        2   
4        7.95          Visa        5   

                                        product_info  
0  {'asin': '0700026657', 'title': 'Anno 2070', '...  
1  {'asin': '0700026657', 'title': 'Anno 2070', '...  
2  {'asin': '0700026657', 'title': 'Anno 2070', '...  
3  {'asin': '0700026657', 'title': 'Anno 2070', '...  
4  {'asin': '0700026657', 'title': 'Anno 2070

In [7]:
merged_df.shape

(6453130, 8)

In [8]:
output_file = "G:\\DSA5104 Project\\generated data\\transaction_data_t.json"
chunk_size = 1000  

with open(output_file, 'w', encoding='utf-8') as file:
    for i in range(0, len(merged_df), chunk_size):
        chunk = merged_df.iloc[i:i + chunk_size]
        chunk_json = chunk.to_json(orient='records', lines=True)
        file.write(chunk_json)
        file.write("\n")